In [44]:
#amalgamate the data, assuming all data is in a folder Raw_Data
import json
current_season = 2000
end_season = 2017

all_data = [] #where all play data will reside

while current_season <= end_season: 
    filename = 'Raw_Data/' + str(current_season) + '-' + str(current_season + 1) + '.json'
    print(filename)
    with open(filename) as data_file:    
        data = json.load(data_file)
        all_data.append(data)
    
    current_season += 1

Raw_Data/2000-2001.json
Raw_Data/2001-2002.json
Raw_Data/2002-2003.json
Raw_Data/2003-2004.json
Raw_Data/2004-2005.json
Raw_Data/2005-2006.json
Raw_Data/2006-2007.json
Raw_Data/2007-2008.json
Raw_Data/2008-2009.json
Raw_Data/2009-2010.json
Raw_Data/2010-2011.json
Raw_Data/2011-2012.json
Raw_Data/2012-2013.json
Raw_Data/2013-2014.json
Raw_Data/2014-2015.json
Raw_Data/2015-2016.json
Raw_Data/2016-2017.json
Raw_Data/2017-2018.json


In [45]:
#map of each team name to the organization id its associated with
team_names_to_organization_id_map = {
 'Atlanta Hawks': 1,
 'Boston Celtics': 2,
 'Chicago Bulls': 3,
 'Cleveland Cavaliers': 4,
 'Dallas Mavericks': 5,
 'Denver Nuggets': 6,
 'Detroit Pistons': 7,
 'Golden State Warriors':8,
 'Houston Rockets': 9,
 'Indiana Pacers': 10,
 'Los Angeles Clippers': 11,
 'Los Angeles Lakers': 12,
 'Miami Heat': 13,
 'Milwaukee Bucks':14,
 'Minnesota Timberwolves':15,
 'New York Knicks':16,
 'Orlando Magic':17,
 'Philadelphia 76ers':18,
 'Phoenix Suns':19,
 'Portland Trail Blazers':20,
 'Sacramento Kings':21,
 'San Antonio Spurs':22,
 'Toronto Raptors':23,
 'Utah Jazz':24,
 'Washington Wizards':25,
 'Brooklyn Nets': 26,
 'New Jersey Nets': 26,
 'Charlotte Bobcats': 27,
 'Charlotte Hornets': 27,
 'Memphis Grizzlies': 28,
 'Vancouver Grizzlies': 28,
 'Oklahoma City Thunder' : 29,
 'Seattle SuperSonics' : 29,
 'New Orleans/Oklahoma City Hornets': 30,
 'Oklahoma City Hornets': 30,
 'New Orleans Pelicans': 30,
 'New Orleans Hornets' : 30
}

In [46]:
#adding home_org_id and away_org_id to each game as specified in team_names_to_organization_id_map
for season in all_data:
    for game in season:
        teams_and_data = game["teams_and_data"]
        at = teams_and_data.index(" at ")
        pbp = teams_and_data.index("Play-By-Play")
        home_team = teams_and_data[at+4: pbp].rstrip()
        away_team = teams_and_data[0:at].rstrip()
        home_org_id = team_names_to_organization_id_map[home_team]
        away_org_id = team_names_to_organization_id_map[away_team]
        game["home_org_id"] = home_org_id
        game["away_org_id"] = away_org_id
        game.pop("teams_and_data", None) #get rid of unecessary string representation of data


In [47]:
#parsing and adding keys (ints) home_score, away_score for each play. takes 30-40 seconds.
for season in all_data:
    for game in season:
        last_away_score = "0"
        last_home_score = "0"
        for play in game["plays"]:
            
            #manipulating score for all plays
            score_text = play["score"]
            if score_text:
                hyphen = score_text.index("-")
                last_away_score = score_text[0:hyphen]
                last_home_score = score_text[hyphen + 1:]
            play["home_score"] = int(last_home_score)
            play["away_score"] = int(last_away_score)
            play.pop("score", None) #get rid of unecessary string representation of data
            
            #manipulating time, only for plays that already have time
            time_text = play["time"]
            if time_text:
                colon = time_text.index(":")
                period = time_text.index(".")
                minutes = int(time_text[0:colon])
                seconds = int(time_text[colon+1: period])
                tenths_seconds = int(time_text[period+1:])
                
                remaining_seconds_in_period = 60 * minutes + seconds + .1 * tenths_seconds
                play["rem_secs_in_period"] = remaining_seconds_in_period
                play.pop("time", None)           

In [50]:
#check out example of game/play representation
example_season = 0
example_game = 0
all_data[example_season][example_game]

{'time_and_location': '\n7:30 PM, October 31, 2000Philips Arena, Atlanta, Georgia\nLogos via Sports Logos.net\n            / About logos\n',
 'plays': [{'home_event': 'Jump ball: A. Henderson vs. E. Campbell (J. Jackson gains possession)',
   'quarter': 1,
   'away_event': 'Jump ball: A. Henderson vs. E. Campbell (J. Jackson gains possession)',
   'home_score': 0,
   'away_score': 0,
   'rem_secs_in_period': 720.0},
  {'quarter': 1,
   'home_event': 'J. Jackson misses 2-pt jump shot from 8 ft',
   'away_event': '\xa0',
   'home_score': 0,
   'away_score': 0,
   'rem_secs_in_period': 707.0},
  {'quarter': 1,
   'home_event': 'H. Mottola makes 2-pt tip-in at rim',
   'away_event': '\xa0',
   'home_score': 2,
   'away_score': 0,
   'rem_secs_in_period': 706.0},
  {'quarter': 1,
   'home_event': 'Offensive rebound by H. Mottola',
   'away_event': '\xa0',
   'home_score': 2,
   'away_score': 0,
   'rem_secs_in_period': 706.0},
  {'quarter': 1,
   'home_event': '\xa0',
   'away_event': 'B. D